In [8]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# I/O -----------------------------------

input_features_path1 = '/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/features/marker_abundance_t2d_rf_forfeatures.txt'
input_features_path2 = '/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/features/abundance_t2d_rf_forfeatures.txt'


In [10]:
features1 = pd.read_csv(input_features_path1, sep='\t', header=None, index_col=0, dtype=unicode)
print features1
features1=features1.iloc[0:100,0:1]

features2 = pd.read_csv(input_features_path2, sep='\t', header=None, index_col=0, dtype=unicode)
#print features2
features2=features2.iloc[0:50,0:1]


                                                        1                  2  \
0                                                                              
0        gi|522837947|ref|NZ_KE159513.1|:c2116911-2116363  0.000609737281763   
1           gi|423335209|ref|NZ_JH976498.1|:329186-330046  0.000527046944886   
2           gi|479146200|ref|NC_021012.1|:2561560-2562087  0.000525042986288   
3             gi|522837954|ref|NZ_KE159520.1|:82176-83867  0.000492790695507   
4           gi|479146200|ref|NC_021012.1|:1734187-1734615  0.000460337220445   
5          gi|301309650|ref|NZ_GG774987.1|:c983767-982835  0.000440094917077   
6            gi|242362090|ref|NZ_GG692728.1|:c35627-34836  0.000433670957033   
7           gi|522837953|ref|NZ_KE159519.1|:114482-115072  0.000421160712701   
8        gi|498503065|ref|NZ_KB822571.1|:c1270373-1269822  0.000401639777366   
9          gi|498503065|ref|NZ_KB822571.1|:c399870-399277  0.000393137305166   
10       gi|522837953|ref|NZ_KE159519.1|

In [124]:
input_data_path1 = '/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/data/marker_abundance-WT2D.txt'
output_results_path1 = '/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/results/after_feature_selection_marker_abundance-WT2D'

input_data_path2 = '/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/data/abundance-WT2D.txt'
output_results_path2 = '/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/results/after_feature_selection_abundance-WT2D'


output_results_path =  '/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/finalresults/combined-WT2D'



disease_name = 't2d'
classifier = 'rf'
runs = 10
cv_folds = 20
feature_identifier_list = ['gi','k__']


In [125]:
class class_metrics:
	def __init__(self):
		self.accuracy = []
		self.f1 = []
		self.precision = []
		self.recall = []
		self.auc = []
		self.roc_curve = []
		self.confusion_matrix = []

class feature_importance:
	def __init__(self, feat, p):
		self.feat_sel = feat
		self.imp = np.array([p]*len(feat))

class class_params:
	def __init__(self):
		self.learner_type = []
		self.feature_selection = []
		self.cv_folds = []
		self.cv_grid = []
		self.cv_scoring = []
		self.fs_grid = []


In [126]:
def save_results(l, l_es, p_es, i_tr, i_u, nf, runs, cv_folds):
	n_clts = len(np.unique(l))
	cm = class_metrics()

	for j in range(runs*cv_folds):
		l_ = pd.DataFrame([l.loc[i] for i in l[~i_tr[j] & i_u[j/cv_folds]].index]).values.flatten().astype('int')
		l_es_ = l_es[j].values.flatten().astype('int')
		if (lp.learner_type == 'rf') | (lp.learner_type == 'svm'):
			p_es_pos_ = p_es[j].loc[:,1].values
		else:
			p_es_pos_ = p_es[j].loc[:,0].values
		ii_ts_ = [i for i in range(len(i_tr[j])) if i_tr[j][i]==False]

		cm.accuracy.append(metrics.accuracy_score(l_, l_es_))
		cm.f1.append(metrics.f1_score(l_, l_es_, pos_label=None, average='weighted'))
		cm.precision.append(metrics.precision_score(l_, l_es_, pos_label=None, average='weighted'))
		cm.recall.append(metrics.recall_score(l_, l_es_, pos_label=None, average='weighted'))
		if len(np.unique(l_)) == n_clts:
			if n_clts == 2:
				cm.auc.append(metrics.roc_auc_score(l_, p_es_pos_))
				cm.roc_curve.append(metrics.roc_curve(l_, p_es_pos_))
			cm.confusion_matrix.append(metrics.confusion_matrix(l_, l_es_, labels=np.unique(l.astype('int'))))

	fidout.write('#samples\t' + str(sum(sum(i_u))/len(i_u)))
	fidout.write('\n#features\t' + str(nf))
	fidout.write('\n#runs\t' + str(runs))
	fidout.write('\n#cv_folds\t' + str(cv_folds))

	fidout.write('\naccuracy\t' + str(np.mean(cm.accuracy)) + '\t' + str(np.std(cm.accuracy)))
	fidout.write('\nf1\t' + str(np.mean(cm.f1)) + '\t' + str(np.std(cm.f1)))
	fidout.write('\nprecision\t' + str(np.mean(cm.precision)) + '\t' + str(np.std(cm.precision)))
	fidout.write('\nrecall\t' + str(np.mean(cm.recall)) + '\t' + str(np.std(cm.recall)))
	if n_clts == 2:
		fidout.write('\nauc\t' + str(np.mean(cm.auc)) + '\t' + str(np.std(cm.auc)))
	else:
		fidout.write('\nauc\t[]\t[]')
	fidout.write('\nconfusion matrix')
	if len(cm.confusion_matrix) > 0:
		for i in range(len(cm.confusion_matrix[0])):
			for i2 in range(len(cm.confusion_matrix[0][i])):
				fidout.write('\t' + str(np.sum([cm.confusion_matrix[j][i][i2] for j in range(len(cm.confusion_matrix))])))
			fidout.write('\n')
	else:
		fidout.write('\n')

	return cm

def set_class_params(classifier):
	lp = class_params()

	lp.learner_type = classifier
	lp.cv_folds = 5

	if lp.learner_type == 'rf':
		lp.cv_grid = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200]
	elif lp.learner_type == 'svm':
		lp.cv_grid = [ {'C': [1, 10, 100, 1000], 'kernel': ['linear']}, {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']} ]
	elif lp.learner_type == 'lasso':
		lp.cv_grid = [np.logspace(-4, -0.5, 50)]
	elif lp.learner_type == 'enet':
		lp.cv_grid = [np.logspace(-4, -0.5, 50), [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0]]
	elif lp.learner_type == 'bayesrr':
		loglist = list(np.logspace(-7, -0.5, 10))
		lp.cv_grid = [{'alpha_1': [loglist[0]], 'alpha_2': [loglist[-1]], 'lambda_1': loglist,'lambda_2': loglist}]
	elif lp.learner_type == 'xgb':
	 	lp.cv_grid=[{'min_child_weight':[1,2,3,4],'gamma':[0,0.1,0.5,1],'learning_rate':[0.05,0.1,0.15,0.01],'max_depth':[4,5,6,8,10,12],'n_estimators':[10,75,100,125,150,500]}]
	if lp.learner_type == 'svm' or lp.learner_type == 'bayesrr' or lp.learner_type == 'xgb':
		lp.cv_scoring = 'accuracy'
	return lp


In [138]:
data1 = pd.read_csv(input_data_path1, sep='\t', header=None, index_col=0, dtype=unicode).T
data2 = pd.read_csv(input_data_path2, sep='\t', header=None, index_col=0, dtype=unicode).T


labels = pd.DataFrame([0]*len(data1))
labels.iloc[data1.index[data1['disease']=='t2d']] = 1
# print labels
# print data1
left1=data1.iloc[:,0:2]
right1=data1.iloc[:,2:]
print(right1)
features1.to_csv("/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/check1.csv")

features1_list = features1[1].tolist()

right_selected1=right1[features1_list]
# print right_selected
# print features1
# print right
data1=pd.concat([left1, right_selected1], axis=1)
# print data1

0  gi|104773257|ref|NC_008054.1|:116729-117526  \
1                                            0   
2                                  3.759398496   
3                                            0   
4                                            0   
5                                            0   
6                                            0   
7                                            0   
8                                            0   
9                                            0   
10                                           0   
11                                           0   
12                                           0   
13                                 1.253132832   
14                                           0   
15                                 7.518796992   
16                                           0   
17                                           0   
18                                           0   
19                                           0   


In [128]:
left2=data2.iloc[:,0:2]
right2=data2.iloc[:,2:]
features2.to_csv("/Users/sonali/Desktop/M229S/CibioCM-metaml-56b1efc1da6d/check2.csv")

features2_list = features2[1].tolist()

right_selected2=right2[features2_list]
# print right_selected
# print features1
# print right
data=pd.concat([data1, right_selected2], axis=1)

In [129]:

cross_valid_data = pd.DataFrame(True, index=range(len(data.index)), columns=range(runs*cv_folds))
run_data = pd.DataFrame(True, index=range(len(data.index)), columns=range(runs))

col = range(len(data2.index))
# print cross_valid_data
# print run_data
# print col

In [130]:
for j in range(runs):
	fold_indices = StratifiedKFold(labels.iloc[run_data.values.T[j],0], cv_folds, shuffle=True)

	for i in range(cv_folds):
		for index in np.where(fold_indices.test_folds == i)[0]:
			cross_valid_data.iloc[col[index], j * cv_folds + i] = False

cross_valid_data = cross_valid_data.T.values
run_data = run_data.T.values

feat = [col_name for col_name in data.columns if sum([feature_identifier in col_name for feature_identifier in feature_identifier_list])>0]
if 'unclassified' in data.columns:
	feat.append('unclassified')

print len(feat)

150


In [131]:
data = data.loc[:,feat].astype('float')
data = (data - data.min())/(data.max() - data.min())

clf = []
feature_importance_list = []
prob_estimate = []
label_estimate = []
lp = set_class_params(classifier)



In [132]:
for j in range(runs*cv_folds):
    feature_importance_list.append(feature_importance(feat, 1.0/len(feat)))
    if lp.learner_type == 'rf':
        clf.append(RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2, n_jobs=-1).\
        fit(data.loc[cross_valid_data[j] & run_data[j/cv_folds], \
        feature_importance_list[j].feat_sel].values, labels[cross_valid_data[j] & run_data[j/cv_folds]].values.flatten().astype('int')))
    
    if (lp.learner_type == 'rf') | (lp.learner_type == 'svm'):
        prob_estimate.append(pd.DataFrame(clf[j].predict_proba(data.loc[~cross_valid_data[j] & run_data[j/cv_folds],\
        feature_importance_list[j].feat_sel].values)))
        label_estimate.append(pd.DataFrame([list(prob_estimate[j].iloc[i,:]).\
        index(max(prob_estimate[j].iloc[i,:])) for i in range(len(prob_estimate[j]))]))
    print j,"---------"

    
fidout = open(output_results_path + '.txt','w')

class_metric_object = save_results(labels, label_estimate, prob_estimate, cross_valid_data, run_data, len(feat), runs,cv_folds)

fidout.close()

0 ---------
1 ---------
2 ---------
3 ---------
4 ---------
5 ---------
6 ---------
7 ---------
8 ---------
9 ---------
10 ---------
11 ---------
12 ---------
13 ---------
14 ---------
15 ---------
16 ---------
17 ---------
18 ---------
19 ---------
20 ---------
21 ---------
22 ---------
23 ---------
24 ---------
25 ---------
26 ---------
27 ---------
28 ---------
29 ---------
30 ---------
31 ---------
32 ---------
33 ---------
34 ---------
35 ---------
36 ---------
37 ---------
38 ---------
39 ---------
40 ---------
41 ---------
42 ---------
43 ---------
44 ---------
45 ---------
46 ---------
47 ---------
48 ---------
49 ---------
50 ---------
51 ---------
52 ---------
53 ---------
54 ---------
55 ---------
56 ---------
57 ---------
58 ---------
59 ---------
60 ---------
61 ---------
62 ---------
63 ---------
64 ---------
65 ---------
66 ---------
67 ---------
68 ---------
69 ---------
70 ---------
71 ---------
72 ---------
73 ---------
74 ---------
75 ---------
76 ---------
77 ------